# Experiment Data Collection
### Running the Experiments

The method ```create_network``` begins with a specified graph, and adds nodes one by one. using the function ```create_edge```.
With each node added, edges are also added according to specified rules (*binomial / scale-free / directed binomial / directed scale-free*). 

In [55]:
import networkx as nx
import random

# Setting seed.
random.seed(101)

# Binomial edge allocation probability.
binomial_probability = 0.1

# Setting recursion rules
recursion_level = 0
recursion_threshold = 100

# Edge creation function.
def create_edge(G, i, j, capacity, changes, edge_reallocation_algorithm, directed):
    G.add_edge(i, j)  # Add an edge between the nodes.
    changes.append(('add', i, j)) # Record changes for visualisation.
    if directed:
        if G.in_degree(j) > capacity:
            edge_reallocation = reallocate_edge(G, j, capacity, changes, edge_reallocation_algorithm, directed)
    else:
        if G.degree(j) > capacity:  # If over capacity, remove node's edges.
            edge_reallocation = reallocate_edge(G, j, capacity, changes, edge_reallocation_algorithm, directed)
            if not edge_reallocation:
                return False
    return True

# Edge reallocation algorithm.
def reallocate_edge(G, node, capacity, changes, edge_reallocation_algorithm, directed):
    global recursion_level
    global recursion_threshold
    neighbors = list(G.neighbors(node))
    predecessors = list(G.predecessors(node)) if directed else None
    for neighbor in neighbors:
        if G.has_edge(neighbor, node):
            changes.append(('remove', neighbor, node))  # Record changes for visualisation.
            G.remove_edge(node, neighbor)  # Remove each edge individually
    if directed:
        node_list = predecessors
    else:
        node_list = neighbors
        # Iterate through the former neighbors.
        for node in node_list:
            potential_nodes = [n for n in G.nodes if not G.has_edge(node, n) and n != node]
            if edge_reallocation_algorithm == 'binomial':
                selected_node = random.choice(potential_nodes)
            elif edge_reallocation_algorithm == 'scale-free':
                degrees = [G.in_degree(n) for n in potential_nodes] if directed else [G.degree(n) for n in potential_nodes]
                selected_node = random.choices(potential_nodes, weights=degrees, k=1)[0]
            else:
                selected_node = random.choice(potential_nodes)
            recursion_level += 1
            if recursion_level > recursion_threshold:
                return False
            create_edge(G, node, selected_node, capacity, changes, edge_reallocation_algorithm, directed)
    return True

# Network Creation
def create_network(n_nodes, capacity, edge_allocation_algorithm, edge_reallocation_algorithm, directed):
    global recursion_level
    global binomial_probability
    G_init = nx.DiGraph() if directed else nx.Graph()
    G = G_init.copy()
    changes = []
    for i in range(n_nodes):
        G.add_node(i)
        if i > 1:
            for j in range(i - 1):
                if edge_allocation_algorithm == 'binomial':
                    probability = binomial_probability
                elif edge_allocation_algorithm == 'scale-free':
                    degrees = [G.in_degree(n) for n in G.nodes] if directed else [G.degree(n) for n in G.nodes]
                    total_degree = sum(degrees)
                    probability = degrees[j]/total_degree if total_degree > 0 else 0
                else:
                    probability = binomial_probability
                if random.random() < probability:
                    edge_creation = create_edge(G, i, j, capacity, changes, edge_reallocation_algorithm, directed)
                    if not edge_creation:
                        print('Ending experiment (recursion will likely be infinite).')
                        return G_init, changes

    print('Ending experiment (network size reached).')
    return G_init, changes

### Writing the Changes to GEXF files.

The create_network function returns the initial graph and a list of changes made to it.
These are subsequently stored in a GEXF file.

In [56]:
def write_to_gexfs(G_init, changes, path):
    G = G_init.copy()  # Make a copy of the initial graph
    for timestep, change in enumerate(changes):
        operation, i, j = change
        if operation == 'add':
            G.add_edge(i, j)
        elif operation == 'remove':
            G.remove_edge(i, j)
        nx.write_gexf(G, f"{path}/timestep_{timestep}.gexf") # type: ignore

In [57]:
import os

def run_experiment(n, capacity, parameters):
    # edge_allocation_algorithm, edge_reallocation_algorithm, directed, name
    edge_allocation_algorithm = parameters[0]
    edge_reallocation_algorithm = parameters[1]
    directed = parameters[2]
    name = parameters[3]
    G_init, changes = create_network(n, capacity, edge_allocation_algorithm, edge_reallocation_algorithm, directed)
    path = f'/Users/aidanlowrie/Documents/GitHub/MRWPProject/Aidan_stuff/experiment_data/{name}'
    os.makedirs(path, exist_ok=True)
    write_to_gexfs(G_init, changes, path)

In [61]:
# Not varying n or capacity. Varying algorithms and directedness.
n = 100
capacity = 10

experiments = { 0:['binomial', 'binomial', 'undirected', 'undirected_binomial_graph_binomial_reallocation'], 
          1: ['binomial', 'scalefree', 'undirected', 'undirected_binomial_graph_scalefree_reallocation'],
          2: ['scalefree', 'binomial', 'undirected', 'undirected_scalefree_graph_binomial_reallocation'],
          3: ['scalefree', 'scalefree', 'undirected', 'undirected_scalefree_graph_scalefree_reallocation'],
          4:['binomial', 'binomial', 'directed', 'directed_binomial_graph_binomial_reallocation'], 
          5: ['binomial', 'scalefree', 'directed', 'directed_binomial_graph_scalefree_reallocation'],
          6: ['scalefree', 'binomial', 'directed', 'directed_scalefree_graph_binomial_reallocation'],
          7: ['scalefree', 'scalefree', 'directed', 'directed_scalefree_graph_scalefree_reallocation']}

for i in range(len(experiments.items())):
    run_experiment(n, capacity, experiments[i])

Ending experiment (network size reached).
Ending experiment (network size reached).
Ending experiment (network size reached).
Ending experiment (network size reached).
Ending experiment (network size reached).
Ending experiment (network size reached).
Ending experiment (network size reached).
Ending experiment (network size reached).
